In [2]:
import torch
import pandas as pd 
import numpy as np

🔹 What’s left / next steps

Class distribution / skewness Done 

Check label counts → compute % per class Done

Merge similar attacks (e.g., SSH + FTP Bruteforce → “Bruteforce”) Done

Feature scaling

Neural networks are sensitive to numeric ranges

Apply StandardScaler or MinMaxScaler to all numeric columns

Categorical encoding

Protocol → one-hot or label encoding

Dst_Port → optional: one-hot top N ports, merge rare ports into “other”

Train/test split

Stratified by Label

Recommended: 80% train / 20% test

Class imbalance handling

Optional but recommended:

Use class weights in NN loss function

Or oversample rare attack classes

Optional feature engineering / selection

Check correlation → remove extremely redundant features

Create engineered temporal features from Timestamp if desired (hour, day of week)

Final sanity checks

No NaNs, inf, or placeholder negatives remain

All features are numeric and scaled

Categorical features encoded

Labels merged/encoded and consistent

In [3]:
df=  pd.read_csv(r"/root/Cyber/Cyber_security/datasets/02-14-2018_uncleaned.csv")

In [4]:
df.Label.value_counts(normalize=True)*100


Label
benign            63.544031
bruteforce        18.503838
ssh-bruteforce    17.952131
Name: proportion, dtype: float64

In [5]:
df["Label"]=df["Label"].replace(["ssh-bruteforce"],"bruteforce")

In [6]:
df.Label.value_counts(normalize=True)*100


Label
benign        63.544031
bruteforce    36.455969
Name: proportion, dtype: float64

WE WILL USE STANDARD SCALER BECAUSE MOST OF  THE PAPERS HAVE USED STANDARD SCALER WHICH USES Z SCORE NORMALIZATION
AND BECAUSE NETWORK TRAFFIC CAN HAVE VERY HIGH OUTLIERS AND MIN MAX SQUASHES THEM INTO A TINY RANGE , BEFORE SCALING WE NEED TO SPLIT THE DATA INTO TRAIN TEST SPLIT DUE TO DATA LEAKAGE

In [7]:
K = 10
top_ports = df["Dst_Port"].value_counts().nlargest(K).index

df["Dst_Port"] = df["Dst_Port"].where(
    df["Dst_Port"].isin(top_ports),
    "Other"
)

df["Dst_Port"] = df["Dst_Port"].astype(str)


In [8]:
print(df["Dst_Port"].value_counts())

Dst_Port
53       198117
21       193422
22       187881
80       129625
Other     93741
443       92094
3389      90606
445       40304
0         11876
5355       5083
137        2191
Name: count, dtype: int64


In [9]:
df["Label"]=df["Label"].str.strip().str.lower()

In [10]:
df["Label"].value_counts()

Label
benign        663997
bruteforce    380943
Name: count, dtype: int64

In [11]:
df['Label_encoded'] = df['Label'].map({'benign': 0, 'bruteforce': 1})

In [12]:
df['Label']=df["Label_encoded"]
df.drop(columns=["Label_encoded"],inplace=True)
df["Label"].value_counts()

Label
0    663997
1    380943
Name: count, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer


In [14]:
onehot_cols = ["Protocol", "Dst_Port"]

numeric_cols = df.drop(columns=["Label", "Protocol", "Dst_Port"]).columns


In [15]:


preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("cat", OneHotEncoder(
            drop="first",          # avoids dummy trap
            handle_unknown="ignore",
            sparse_output=False
        ), onehot_cols)
    ],
    remainder="drop"
)


In [16]:
X = df.drop(columns=["Label"])
y = df["Label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)


In [17]:
#Sanity check
df["Dst_Port"].apply(type).value_counts()


Dst_Port
<class 'str'>    1044940
Name: count, dtype: int64

In [23]:
print(type(X_train_proc))
type(X_test_proc)

<class 'numpy.ndarray'>


numpy.ndarray

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, precision_score, recall_score


Checklist 

Understand & Prepare Data ✔️

Collect dataset, clean & preprocess ✔️

Split into train/validation/test sets  ✔️

Normalize/scale features ✔️

Batch the data

Define Model Architecture

Decide number & type of layers

Choose activation functions

Think about input & output shapes

Choose Loss Function & Optimizer

Pick a loss suitable for task (classification/regression)

Select optimizer (Adam, SGD, etc.)

Set learning rate & other hyperparameters

Train the Model

Loop over epochs and batches

Forward pass → compute loss → backward pass → update weights

Monitor training loss & metrics

Validate & Evaluate

Test on unseen data

Calculate accuracy, F1, RMSE, or other metrics

Adjust model or hyperparameters if needed

Save & Deploy

Save model weights

Load model for inference or further training

Iterate & Improve

Tune hyperparameters

Experiment with architecture

Handle overfitting (dropout, regularization)

Scale up data if possible

In [25]:
y_train1=y_train.to_numpy()
y_test1=y_test.to_numpy()

In [52]:
X_train=torch.from_numpy(X_train_proc)
X_test=torch.from_numpy(X_test_proc)
Y_test=torch.from_numpy(y_test1)
Y_train=torch.from_numpy(y_train1)
X_train=torch.tensor(X_train,dtype=torch.float32)
X_test=torch.tensor(X_test,dtype=torch.float32)
Y_test=torch.tensor(Y_test , dtype=torch.float32)
Y_train=torch.tensor(Y_train , dtype=torch.float32)

print(X_train)
print(X_test)
print(Y_train)
print(Y_test)


/tmp/ipykernel_809/1397678349.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train=torch.tensor(X_train,dtype=torch.float32)
/tmp/ipykernel_809/1397678349.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test=torch.tensor(X_test,dtype=torch.float32)


tensor([[-0.3221, -0.1169, -0.0593,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3234, -0.0722, -0.0309,  ...,  0.0000,  1.0000,  0.0000],
        [-0.3228, -0.1169, -0.0593,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.3234, -0.1169, -0.0593,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3234, -0.1169, -0.0593,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3235, -0.0945, -0.0593,  ...,  0.0000,  0.0000,  1.0000]])
tensor([[-2.4156e-01,  3.9714e-02, -2.5028e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-3.2346e-01, -1.1692e-01, -5.9349e-02,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-3.2346e-01, -1.1692e-01, -5.9349e-02,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 4.0432e+00,  2.1872e-01,  6.3818e-02,  ...,  0.0000e+00,
          1.0000e+00,  0.0000e+00],
        [ 1.9700e+00,  1.9634e-01,  7.3292e-02,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-3.2346e-01, -1.1692e-01, -5.

/tmp/ipykernel_809/1397678349.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_test=torch.tensor(Y_test , dtype=torch.float32)
/tmp/ipykernel_809/1397678349.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_train=torch.tensor(Y_train , dtype=torch.float32)


In [53]:
from torch.utils.data import TensorDataset

num_epoch=30
batch_size=1024
lr= 1e-3
input_dim=X_train.shape[1]

train_dataset = TensorDataset(X_train, Y_train)
test_dataset  = TensorDataset(X_test, Y_test)


test_dataloader=DataLoader(test_dataset,batch_size=batch_size,shuffle=True)
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=False)

In [54]:
train_features,train_labels=next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([1024, 70])
Labels batch shape: torch.Size([1024])


In [55]:
#Compute class weights and use in loss function 
num_pos=(Y_train==1).sum()
num_neg=(Y_train==0).sum()
pos_weight=num_neg/num_pos

In [56]:
criterion= nn.BCEWithLogitsLoss(pos_weight=pos_weight)

Baseline Model Architecture

Input: n_features

Input Layer

Linear(n_features → 128)

BatchNorm1d(128)

ReLU

Dropout(p = 0.3)

Hidden Layer 1

Linear(128 → 64)

BatchNorm1d(64)

ReLU

Dropout(p = 0.3)

Hidden Layer 2

Linear(64 → 32)

ReLU

Output Layer

Linear(32 → 1)



In [57]:
class BruteForceNN(nn.Module):
    """
    Uses 2 Hidden layers with dropout = 0.03 and BatchNorm1d
    Relu as an activation function and loss is BCELogitLoss with class weights


    """
    def __init__(self,input_dim):
        super().__init__()
        self.model=nn.Sequential(
            #Input layer
            nn.Linear(input_dim,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.03),

            #Hidden layer 

            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.03),

            #Hidden layer 2
            nn.Linear(64,32),
            nn.ReLU(),

            #Output layer
            nn.Linear(32,1)            
        )
    def forward(self,x):
        return self.model(x)
        


In [58]:
model=BruteForceNN(input_dim=input_dim)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [59]:
device=None
if torch.cuda.is_available():
    device="cuda"
else:
    device="cpu"

In [60]:
X_train.dtype

torch.float32

In [62]:
best_val_loss = float("inf")  


for epoch in range(num_epoch):

    # ----- TRAINING -----
    model.train()
    running_loss = 0.0

    for X_batch, Y_batch in train_dataloader:
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, Y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(train_dataloader)  # after loop

    # ----- VALIDATION -----
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X_batch, Y_batch in test_dataloader:
            X_batch = X_batch.to(device).float()
            Y_batch = Y_batch.to(device).float().unsqueeze(1)

            logits = model(X_batch)
            loss = criterion(logits, Y_batch)
            val_loss += loss.item()

            preds = torch.sigmoid(logits)
            all_preds.append(preds.cpu())
            all_labels.append(Y_batch.cpu())

    val_loss = val_loss / len(test_dataloader)  # after loop
    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()
    all_preds_class = (all_preds > 0.5).astype(int)

    # ----- METRICS -----
    roc_auc = roc_auc_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds_class)
    recall = recall_score(all_labels, all_preds_class)

    # ----- SAVE BEST MODEL -----
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pt")

    print(f"Epoch {epoch+1}/{num_epoch} | "
          f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
          f"ROC-AUC: {roc_auc:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")


Epoch 1/30 | Train Loss: 0.0002 | Val Loss: 0.0002 | ROC-AUC: 1.0000 | Precision: 0.9999 | Recall: 1.0000
Epoch 2/30 | Train Loss: 0.0001 | Val Loss: 0.0001 | ROC-AUC: 1.0000 | Precision: 0.9999 | Recall: 1.0000
Epoch 3/30 | Train Loss: 0.0001 | Val Loss: 0.0001 | ROC-AUC: 1.0000 | Precision: 0.9999 | Recall: 1.0000
Epoch 4/30 | Train Loss: 0.0001 | Val Loss: 0.0000 | ROC-AUC: 1.0000 | Precision: 1.0000 | Recall: 1.0000
Epoch 5/30 | Train Loss: 0.0001 | Val Loss: 0.0001 | ROC-AUC: 1.0000 | Precision: 1.0000 | Recall: 1.0000
Epoch 6/30 | Train Loss: 0.0002 | Val Loss: 0.0005 | ROC-AUC: 1.0000 | Precision: 0.9993 | Recall: 1.0000
Epoch 7/30 | Train Loss: 0.0001 | Val Loss: 0.0000 | ROC-AUC: 1.0000 | Precision: 1.0000 | Recall: 1.0000
Epoch 8/30 | Train Loss: 0.0000 | Val Loss: 0.0001 | ROC-AUC: 1.0000 | Precision: 1.0000 | Recall: 1.0000
Epoch 9/30 | Train Loss: 0.0000 | Val Loss: 0.0002 | ROC-AUC: 1.0000 | Precision: 0.9999 | Recall: 1.0000
Epoch 10/30 | Train Loss: 0.0001 | Val Loss: 0

# Final Remarks

## Summary
We developed a neural network-based IDS specialized for brute-force attacks using the CIC‑IDS 2018 02‑14 dataset. After preprocessing, balancing, and scaling the features, the model achieved near-perfect performance on validation data with ROC-AUC, precision, and recall all ≈1.0.

## Scope & Limitations
This model is attack-specific — it is trained solely on brute-force patterns. It **does not generalize** to other attacks such as DoS, botnets, or reconnaissance unless retrained with those classes. Results reflect dataset simplicity and class separability.

## Future Work
- Extend to a **multi-class IDS** covering other attack types (DoS, botnets, etc.).
- Explore **open-set anomaly detection** to identify unseen attacks.
- Compare neural network performance with traditional ML models like Random Forest or XGBoost.

## Reproducibility
The code, preprocessing pipeline, and training scripts are included. The trained model file is not uploaded due to size constraints but can be generated using the included notebook.
